In [16]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [27]:
input_file = "sheep-data/input.txt"
file = open(input_file, "r") 
text = file.read() 
print('corpus length:', len(text))

corpus length: 100652


In [18]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 90


In [19]:
## Chopping the text up into strings of 40 characters to train on
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 33538


In [20]:
## Vectorizing our sequences
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [21]:
## We're designing our model here- choosing the number of layers and such
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [22]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Showing the poetry quality at the end of each epoch
def on_epoch_end(epoch, logs):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        ## What we're doing is giving the bot 40 characters and asking
        ## it to predict the 41st character, over and over
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [23]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Here's where we train the model on our data
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
33536/33538 [============================>.] - ETA: 0s - loss: 2.5206
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "

But he spoke to re-assure me,
And he"


But he spoke to re-assure me,
And her the shend the sher shen the sher the sher the shend the sher the shend the shen the sher the sher the shen the shend the sher sher the sher the shend the shend the shen whe shend the shend the sher the  and the sher the  hen the shen and the sher the shen the sher sher the  and the shend the sher the sher whe sher the shend the sher the sher the sher the whes the shend the the sher sher and the 
----- diversity: 0.5
----- Generating with seed: "

But he spoke to re-assure me,
And he"


But he spoke to re-assure me,
And he plon the sheat the fou the she shend on the sher ar the shend no  the dor the sou the  han nhit she the bat lon the bler 

A sher a on whe the prens on the when the mer whan the  and the whlos the sher sher the sthe at and b

on countion. 
where you alliuse, lone.


Speads and lest in a gaing at min.

nrougt beer 
Arokiin though and gress and every.

his a eseathy and blave

So Amoving a flace; 
veary in ailAnds when

Lidy tid

lluke, 
There grent ther that fungeoss
Shear histpin to
----- diversity: 1.2
----- Generating with seed: "he man of Uz 
Shall we sit weeping for "
he man of Uz 
Shall we sit weeping for ind
Theig fousuen and mait hrack

Elay, an. wake-, bus it a liosed,

shee il the rove. thin
Alate, neatiag

Your into arood a like
Ake arononed and sabe��� buee,  
ChiTdly lets own-buome they.

ther blact a sadtofaisuy

I and Nard,
Myely whine
shenoting a knownerefters uned: 

Thay: Thrined,
Whainse
:'mathating, ont the holdsidhantent murte!

varm onttrounal and wamy.

th
33538/33538 [==============================] - 59s 2ms/step - loss: 1.7808
Epoch 5/60
33536/33538 [============================>.] - ETA: 0s - loss: 1.6794
----- Generating text after Epoch: 4
----- diversity: 0.2
----- Generating wi

That he hear and some of the rest and ston of the romes lost lost lost lost lost lost lost lost lost lost lost lost lost lost lost lost one lost lost lost lost lost lost lost lost lost lost lost lost lost lost 
----- diversity: 0.5
----- Generating with seed: "p
I could scream all I want
Show you m"
p
I could scream all I want
Show you mire to peep at the sunly who in the ence of the world of you the restering to seen

the searing you and sand and counting to because

I am the same of the world their lost sheeples in the deptong no ever and counting to black hild ow spent in the rock
The soel one desta. That he lear a beece
The say love to sees of the roves of the dight in the fear noter
And see in the even so leep and sca
----- diversity: 1.0
----- Generating with seed: "p
I could scream all I want
Show you m"
p
I could scream all I want
Show you meangers are, or elovelvys,
ghime and nrie to somely laves, as too is by every
agil and muse
loth canges, suptrnges Thitals
Big mounting 
 s

shave helfols, against betond
Onhice
Apone on the threes will the
Sleep of hersend of the revet ti clanrobe, 
Dloon my freees to a oarp,
but us dust learay,
lut herd to to the chistranck a fre'lWh thamp.
Soon't for the strung I ffmilt in the round viomno
Abumy in the kyut of lostinating the wolk
No  the miout to the thing, not emats they the east
But Apled uginous .

The world pri
33538/33538 [==============================] - 65s 2ms/step - loss: 1.2761
Epoch 12/60
33536/33538 [============================>.] - ETA: 0s - loss: 1.2246
----- Generating text after Epoch: 11
----- diversity: 0.2
"---- Generating with seed: "ll fifty lambs lay dead about the hill!
ll fifty lambs lay dead about the hill!

the sheep on the room.

The sheep dread of the day the sheep
That sheep dready of me.

The mild of the house to the round of the world to the steal down the holder, 
The sheep dread of mell

The day, 
The sheep don't be tronged what despurling
That stast has the deep to the sheep
In the ti

fich a past to the house to ear the sheep���


So with the sheep don't fied aseart "pastury fall pass not their some the woldes.


So a sill the sheep the wolde
The out and the earth the sheep.


The becan only real dew


I mire not saams from and t
----- diversity: 1.0
----- Generating with seed: "ose things can't live his eyes
Are open"
ose things can't live his eyes
Are open misht
In faug driptlem.

I wast the sheadges longe.
who with their hidder in sawvork. 

 you a sheep owfur of maging of the Sheep
In the and poncestibut ent eresesprun smy.
I cloul or youggning houses,
The poep over theur footh no )id matuting and more through,

the griant in stand stans
is it would canfonite
Their souts to more shaking thy shall whole, to slail ram. 
They houped an
----- diversity: 1.2
----- Generating with seed: "ose things can't live his eyes
Are open"
ose things can't live his eyes
Are open my last hatter theed
Slap neve sed be taking like you asticary,
Mumps for my smal thing,
Yot a fast ti

...

I don't fisten in the witte af these fueld
Tgats it my sil
33538/33538 [==============================] - 63s 2ms/step - loss: 0.8206
Epoch 44/60
33536/33538 [============================>.] - ETA: 0s - loss: 0.8232
----- Generating text after Epoch: 43
----- diversity: 0.2
----- Generating with seed: " ports everywhere,
but none for to be m"
 ports everywhere,
but none for to be my milds

They wait they leaves, by of the world that sed brother

the man�land of love
And some old the little lambs or I have and day at the witter.

the pats enough of stranged me, 
They wait they leapt a little sheep, now.
Where they bod out of carr of slam to be
till blook 
The Sheep one some blank of slambt him sheep,

noom those a gay.
The sheep trane when the sun out.....

A
----- diversity: 0.5
----- Generating with seed: " ports everywhere,
but none for to be m"
 ports everywhere,
but none for to be my train.
The sheep wath they hearen's stand them!
I wit ey the worn, and sheep, now the truence

othe

Sheep, whos gon, what flut folly if the 
world is the press's stade, broady
Commore nots and got all liove.

I stump strake.
The tarns and shall was itnificicim's need dripe.
Hes  one so was grazing lond of agay, in his nce.

i pull melss comes your every
seep till say say

But deston dorms and promay, 
i attly my clood wide in timan���
there are pastuler. 

So me, one which I was sheep
----- diversity: 1.2
----- Generating with seed: "heep in the field
Violets
Roses
Sheep"
heep in the field
Violets
Roses
Sheep hast have weeplass to ears
He was thicked in the rade.

Sn the atter. thed, ebent court is sheep,tands, is have cindle and marve
With thine, as never beaus every gords,
someth all their meyt
Butt his fly
Ivorch or what close it my lambs lees. 

Forging stimsed!
They go as soul��� 
i am, or good bid tyo happ-and benow hillss
When the nexr kinly would gar���

atis and bellong 
I th
33538/33538 [==============================] - 57s 2ms/step - loss: 0.7952
Epoch 48/60
33536/33538 [

Are as words of the eyes,
And whice is staring the sun pasture.
I will ie silent be a man.
In the world dreads me, 
I with the sheep wind sheep tread, and sheep, not the sleep 
And breather shall began and sheep
----- diversity: 0.5
----- Generating with seed: "ickeyPoet's PagePoemsQuotesCommentsStats"
ickeyPoet's PagePoemsQuotesCommentsStats fleece, black sheep
And in the any time as wolfly aways a B��le
When my free into the rom
Through the sheep in a goot bride.
The sheep is bay to be
belid on so the miggle of 
and pallers
And do wondint is she whole lost like a ready to me, I washer
And I cant my should flowed ret with beep and fleece,

i taking for the rest,

I cant for the rollsw wers.

I am the sheep watching thei
----- diversity: 1.0
----- Generating with seed: "ickeyPoet's PagePoemsQuotesCommentsStats"
ickeyPoet's PagePoemsQuotesCommentsStats tellsing the longer bought 
Are you some sheep in whitens from the excrases opelling Nive.

And time be id they heard.

to everyone a ga

Of the last bitter hour cinningly more magrous this smidon, my flocked,
Ado'l laster limble bosserence
The cath, no past not the haptwree he,
They'le you waits, abe all goose,
Mingly vase flight
undie

Menreamy to dew op hothas stare holfer where,
Inach pool, not must be sweet
darrial !
There' new f.....opise
He' hove
into ease 
id eyest whon graved.
Yet all airy, gravimuled....
head, and over thought as I 
33538/33538 [==============================] - 65s 2ms/step - loss: 0.7463
Epoch 55/60
33536/33538 [============================>.] - ETA: 0s - loss: 0.7442
----- Generating text after Epoch: 54
----- diversity: 0.2
----- Generating with seed: "erything, 
sown fields, some chickens, "
erything, 
sown fields, some chickens, dreaming with apon
Where the skyordered,
I am sheep are bust their sand my elsy the sheep, non��� drest to be the sleep,
And over-prom the sheep are blood,
In the wind the sheep will no mire fashing the sheep,
And then sheep are bust their sace.

Sheecht beling be

As the dried that we peacilading for the hogn when the sheep
That have the gring
I am of it seem to make speak us full dewnward?
What abrel he she been it forget...

self that was stays and dust
when thin, his plowerl gays and desperate the
will night,
In the path our followly, wolves tod........
----- diversity: 1.0
----- Generating with seed: "A mighty bear, massive in size, vast big"
A mighty bear, massive in size, vast big, every strange
There are lift form of stand
It is in tfriens and stalk mayse from the eyeling of breadure
Where the wasckinion liket last here 
Ou ust, meek in the mild
Of libin anoket the lambs

So the plains of spon, that counting sheep
vair one sheepless song, where shalls, herdising tuched
Thou with their fathonchion pire, am I smicf to hell chan, is sweet.

A felt myself man with 
----- diversity: 1.2
----- Generating with seed: "A mighty bear, massive in size, vast big"
A mighty bear, massive in size, vast bighty sumboPries abode. 

. than?s
Gege she meptai

In [26]:
## Here is where you can actually use the model on a given piece of text!

for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = 'Under an angry moon baaaaaaaaaaaaaaaaaaaa'
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(800):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

----- diversity: 0.2
----- Generating with seed: "and theres a post of questionable qualit"
and theres a post of questionable qualiter, worred, herder
there are no dreams. I am the sleep,
To the wolves are sweet.

I feet to me world is the day the work

Sleepy bather the bleet.

I mat eyed to the world,
But don't peanly to the word. 
He wonders if the world,rainion when the sweet

there are no close to be the sleep,
To the plang mell
--feren abrouch
And I am the world with the world,
But will t

/home/Randus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


he sleep, 
Where the poes of the round of the dauting the sheep,
But you beeath will no lied
A sheep know never thousands of sheep,
But will the sheep doging at the world,
But don't pean and the sleep, and white to the word,
My day has heart to be the word.

I sipery the sheep, bot is to be the wolf
It up the restong cannot to sleep towown-
And some milting you,
Worch of the reston't like up with the wo
----- diversity: 0.5
----- Generating with seed: "and theres a post of questionable qualit"
and theres a post of questionable qualitgle of melaying to thine,

I am the sleep,
And I but some coultly now a pashen in the world,
But flow fore for me dist
and bellow sheepylates layded,
Have you arent the world,rain will no greem
The sheep, the sleep,
To the prow the slaw that greans stipred on the bleet.

throust and life we care
And white a white and restand to bleece
The sheep know never thousands of goo in our benteriting the sheep.
wood with heare a spean fleece,

and the child the dung 